# Get the Data
## Download the Data

In [ ]:
import os
import tarfile
import requests as req

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_path=HOUSING_PATH, housing_url=HOUSING_URL):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    # Download the tgz file
    with open(tgz_path, mode="wb") as tarfile_stream:
        response = req.get(housing_url)
        tarfile_stream.write(response.content)
    # Extract the tgz file
    with tarfile.open(tgz_path) as housing_tar:
        housing_tar.extractall(path=housing_path)

In [ ]:
fetch_housing_data()

In [ ]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

## Take a Quick Look at the Data Structure

In [ ]:
housing = load_housing_data()
housing.head()

In [ ]:
housing.info()

In [ ]:
housing["ocean_proximity"].value_counts()

In [ ]:
housing.describe()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
plt.show()

## Create a Test Set

Split train and set test using its own unique identifier, hash it, and check if the hash is lower than or equal to the percentage of the maximum hash value. 
This ensures that the test set will remain consistent across multiple runs, if you refresh the dataset. 
The new test set will contain the percentage of the new instances, but it will not contain any instance that was previously in the training set.
Unfortunately this dataset does not have an identifier column, so we will use row id. 

In [ ]:
from zlib import crc32
import numpy as np

def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

In [ ]:
housing_with_id = housing.reset_index() # adds an 'index' column
housing_with_id["id"] = housing["longitude"] * 1000 + housing["latitude"] 
train_set, test_set = split_train_test_by_id(data=housing_with_id, test_ratio=0.2, id_column="id")

Or you can use train and test split from a library

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

Categorize the housing median income

In [ ]:
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

In [ ]:
housing["income_cat"].hist()

Do stratified sampling with income category

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

Look the income category proportions in the test set

In [ ]:
compare_df = strat_test_set["income_cat"].value_counts() / len(strat_test_set)
compare_df

Look the income category proportions in the full dataset

In [ ]:
housing["income_cat"].value_counts() / len(housing)

Remove the income_cat attribute so the data is back to its original state

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

## Discover and Visualize the Data to Gain Insights

If the training set is very large, you may want to sample an exploration set, to make manipulations easy and fast. In our case, the set is quite small, so you can just work directly on the full set.

Put the test set aside

In [ ]:
housing = strat_train_set.copy()

### Visualizing Geographical Data

Since there is a geographical information (latitude and longitude) in the dataset, we will use scatterplot to visualize the data

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude")

Setting the alpha to 0.1 to make it easier for us to see the plot where the density is high

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

The radius of each circle represents the district’s population (option s), and the color represents the price (option c). 
We will use a predefined color map (option cmap) called jet, which ranges from blue (low values) to red (high prices):

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
                       s=housing["population"] / 100, label="population", figsize=(10, 7),
                       c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True)
plt.legend()

The housing prices are very much related to the location (e.g., close to the ocean) and to the population density

### Looking for Correlations

Since the dataset is not too large, you can easily compute the standard correlation coefficient (also called Pearson’s r) between every pair of attributes using the corr() method:

In [ ]:
corr_matrix = housing.corr(numeric_only=True)

Look at how much each attribute correlates with the median house value

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False)

Another way to check for correlation between attributes is to use pandas scatter_matrix(), which plots every numerical attributes against every numerical attributes.
I just focus on a few promising correlations against median house value

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))

The most promising attribute to predict median house value is median income, let's zoom in on their correlation scatterplot

In [ ]:
housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.1)

The price cap that we noticed earlier is clearly visible as a horizontal line at $500,000. But this plot reveals another horizontal lines.
You have to remove the corresponding districts to prevent your algorithms from learning to reproduce these data quirks.

### Experimenting with Attribute Combinations

One last thing you may want to do before preparing the data for Machine Learning algorithms is to try out various attribute combinations. 
For example, the total number of rooms in a district is not very useful if you don’t know how many households there are. 
What you really want is the number of rooms per household. Similarly, the total number of bedrooms by itself is not very useful: you probably want to compare it to the number of rooms. And the population per household also seems like an interesting attribute combination to look at.

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"] / housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"] / housing["total_rooms"]
housing["populations_per_household"] = housing["population"] / housing["households"]

In [ ]:
corr_matrix = housing.corr(numeric_only=True)
corr_matrix["median_house_value"].sort_values(ascending=False)

## Prepare the Data for Machine Learning Algorithms


Revert to a clean training set (by copying strat_train_set once again). Also separate the predictors and the labels, since we don’t necessarily want to apply the same transformations to the predictors and the target values

In [ ]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()
housing_labels.head()

### Data Cleaning

Most machine learning algorithms can't work with missing features, so we will create a few functions to take care of them.
total_bedrooms attribute has some missing values

We have three options:
1. Get rid of the corresponding districts.
2. Get rid of the whole attribute.
3. Set the values to some value (zero, the mean, the median, etc.).

In [ ]:
housing.dropna(subset=["total_bedrooms"])  # option 1
housing.drop("total_bedrooms", axis=1)  # option 2
median = housing["total_bedrooms"].median() # option 3
housing["total_bedrooms"].fillna(median, inplace=True)

In [ ]:
housing.head()

Or you can also use SimpleImputer to fill missing values for each attribute 

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

Attributes the median can be filled with is numerical. We will get rid of any none numerical columns

In [ ]:
housing_num = housing.drop("ocean_proximity", axis=1)

Fit the median

In [ ]:
imputer.fit(housing_num)

The imputer stores the median on its statistics_ instance

In [ ]:
imputer.statistics_

Now, let's look on the medians of our data 

In [ ]:
housing_num.median().values

Now, we can use this "trained" imputer to fill in our missing values

In [ ]:
X = imputer.transform(housing_num)

The result is a plain NumPy array. Transform it into a dataframe

In [ ]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)

Now let's look at the data again

In [ ]:
housing_tr.info()

### Handling Text and Categorical Attributes

We have only dealing with numerical attributes so fat. Let's take a look at text and categorical attributes

In [ ]:
housing_cat = housing[["ocean_proximity"]]

In [ ]:
housing_cat.value_counts()

It's not an arbitrary text, so we can transform that to a numerical value. Since most machine learning algorithm prefer to work with numerical value, let's convert it to a numerical value.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

Let's look at the list of the categories

In [ ]:
ordinal_encoder.categories_

One issue with this representation is that ML algorithms will assume that two nearby values are more similar than two distant values. 
This may be fine in some cases (e.g., for ordered categories such as “bad,” “average,” “good,” and “excellent”), but it is obviously not the case for the ocean_proximity column (for example, categories 0 and 4 are clearly more similar than categories 0 and 1).

A common solution is to create one binary attribute per category: one attribute equal to 1 when the category is “<1H OCEAN” (and 0 otherwise), another attribute equal to 1 when the category is “INLAND” (and 0 otherwise), and so on.

This is called one-hot encoding, because only one attribute will be equal to 1 (hot), while the others will be 0 (cold). The new attributes are sometimes called dummy attributes.

Scikit-Learn provides a OneHotEncoder class to convert categorical attributes into one-hot encoders.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

Get a list of the categories

In [ ]:
cat_encoder.categories_

### Custom Transformers

Scikit-Learn allows you to create custom transformers by implementing three methods: fit() (returning self), transform(), and fit_transform(). You can inherit from TransformerMixin to get fit_transform() for free. Inheriting from BaseEstimator (and avoiding *args and **kwargs in the constructor) provides get_params() and set_params() methods for hyperparameter tuning. A custom transformer class can seamlessly work with Scikit-Learn functionalities like pipelines, thanks to duck typing.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6


class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_rooms=True):  # no *args and **kwargs
        self.add_bedrooms_per_rooms = add_bedrooms_per_rooms

    def fit(self, X, y=None):
        return self  # nothing else to do

    def transform(self, X, y=None):
        rooms_per_households = X[:, rooms_ix] / X[:, households_ix]
        population_per_households = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_rooms:
            bedrooms_per_rooms = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_households, population_per_households, bedrooms_per_rooms]
        else:
            return np.c_[X, rooms_per_households, population_per_households]
        
attr_adder = CombinedAttributesAdder(add_bedrooms_per_rooms=False)
housing_extra_attribs = attr_adder.transform(housing.values)

The custom transformer has one hyperparameter, add_bedrooms_per_rooms, set to True by default (it is often helpful to set sensible defaults). This hyperparameter will allow you to easily find out whether adding this attribute will helps the Machine Learning or not.

More generally, you can add a hyperparameter to gate any data preparation step that you are not 100% sure about. The more you automate these data preparation steps, the more combinations you can automatically try out, making it much more likely that you will find a great combination (and saving you a lot of time).

### Feature Scaling

With few exceptions, Machine Learning algorithms don’t perform well when the input numerical attributes have very different scales. This is the case for the housing data: the total number of rooms ranges from about 6 to 39,320, while the median incomes only range from 0 to 15. Note that scaling the target values is generally not required.

There are two common ways to get all attributes to have the same scale: minmax scaling and standardization.

Standardization is much less affected by outliers. For example, suppose a district had a median income equal to 100 (by
mistake). Min-max scaling would then crush all the other values from 0–15 down to 0–0.15, whereas standardization would not be much affected

### Transformation Pipelines

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler())
])

housing_num_tr = num_pipeline.fit_transform(housing_num)

The Pipeline constructor takes a list of name/estimator pairs defining a sequence of steps. All but the last estimator must be transformers (i.e., they must have a fit_transform() method). The names can be anything you like (as long as they are unique and don’t contain double underscores, __); they will come in handy later for hyperparameter tuning.

So far, we have handled the categorical and numerical columns separately. It would be more convenient to have a single transformer able to handle all columns. Scikit-Learn in version 0.20 introduced the ColumnTransformer for this purpose. We will use that instead.

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs)
])

housing_prepared = full_pipeline.fit_transform(housing)

In [ ]:
housing_prepared

## Select and Train a Model

### Training and Evaluating on the Training Set

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

In [ ]:
print("Prediction:", lin_reg.predict(some_data_prepared))

In [ ]:
print("Labels:", list(some_labels))

Measure this regression model’s RMSE on the whole training set using Scikit-Learn’s mean_squared_error() function

In [ ]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

Let's use DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

In [ ]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

### Better Evaluation Using Cross-Validation

Scikit-Learn’s cross-validation features expect a utility function (greater is better) rather than a cost function (lower is better), so the scoring function is actually the opposite of the MSE (i.e., a negative value), which is why the preceding code computes -scores before calculating the square root

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [ ]:
display_scores(tree_rmse_scores)

In [ ]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Let's try to use RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)

In [ ]:
housing_predictions = forest_reg.predict(housing_prepared)

In [ ]:
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                                    scoring='neg_mean_squared_error', cv=10)

In [ ]:
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_scores)